In [4]:
import os
import threading
from langchain_aws import ChatBedrock
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from colorama import init, Fore

In [5]:
init(autoreset=True)

In [6]:
MODELS = [
    "meta.llama3-70b-instruct-v1:0", #Big Llama
    "anthropic.claude-3-haiku-20240307-v1:0", #Big Claude
    "anthropic.claude-3-5-sonnet-20240620-v1:0", #Last Claude
    "anthropic.claude-3-sonnet-20240229-v1:0", #Last middle Claude
    "mistral.mixtral-8x7b-instruct-v0:1", #Big Mistral
    "meta.llama3-8b-instruct-v1:0" #Small Llama
]

In [7]:
prompt1 = "Devuelve solo el texto original con la identificación y etiquetación del texto con la información personal del paciente añadiendolo entre claudators, por ejemplo: [**arquitecto**], [**8/9/21**], [**padre**]. No comentes nada más"

# Poner -> Etiquetas 

In [8]:
meddocan = """
Nombres
Datos geográficos
Todos los elementos de las fechas
Números telefónicos
Números FAX
Correos electrónicos
Números de Seguridad Social
Números de registros médicos
Números de beneficiarios del plan de salud
Números de cuenta11.Certificado / números de licencia
Identificadores de vehículos y números de serie, incluidas placas
Identificadores de dispositivo y números de serie
URL web
Direcciones de protocolo de Interne
Identificadores biométricos (es decir, escaneo retiniano, huellas dactilar
Fotos de cara completa e imágenes comparable
Cualquier número de identificación único, característica o código
"""

In [9]:
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are an anonimization tool in identifying attributes in texts that can identify or quasi-identify a user."),
    ("user", "{prompt1} {text} {meddocan}")
])

In [10]:
parser = StrOutputParser()

In [11]:
def get_text_and_masked_carmen(name):
    filename = f'./data/processed/txt/{name}'
    filename_result = f'./data/processed/masked/{name}'
    with open(filename, 'r') as archivo:
        text = archivo.read()

    with open(filename_result, 'r') as archivo:
        text_masked = archivo.read()

    return [text, text_masked]

In [12]:
list_data  = []
counter = 0

In [13]:
import threading
llm_small_llama =   ChatBedrock(  
                            model_id=MODELS[5],
                            region_name='eu-west-2',
                            model_kwargs=dict(temperature=0.1),
                    )
llm_big_llama   =   ChatBedrock(    
                            model_id=MODELS[0],
                            region_name='eu-west-2',
                            model_kwargs=dict(temperature=0.1),
                    )
llm_haiku       =   ChatBedrock(    
                            model_id=MODELS[1],
                            region_name='eu-west-3',
                            model_kwargs=dict(temperature=0.1),
                    )
llm_sonet       =   ChatBedrock(    
                            model_id=MODELS[3],
                            region_name='eu-west-3',
                            model_kwargs=dict(temperature=0.1),
                    )
llm_mistral     =   ChatBedrock(
                        model_id=MODELS[4],
                        region_name='eu-west-2',
                        model_kwargs=dict(temperature=0.1)
                    )

In [14]:
def save_file(filename, text):
    with open(filename, 'w') as archivo:
        archivo.write(text)

In [15]:
def create_folder(name):
    try:
        os.mkdir(name)
        print(f"Folder '{name}' created successfully")
    except FileExistsError:
        pass
    except Exception as e:
        print(f"Error creating the folder '{name}': {e}")


In [16]:
def anonimized(llm=None, name_model=""):
    counter = 0
    path = './data/processed/txt'
    chain = prompt_template | llm | parser
    for filename in os.listdir(path):
        [text, text_hoped] = get_text_and_masked_carmen(filename)
        text_new = chain.invoke({"prompt1":prompt1, "text": text, "meddocan": meddocan})
        create_folder(f'data/anon/raw/{name_model}')
        save_file(f'data/anon/raw/{name_model}/{filename}', text_new)
        counter += 1
        if counter > 4:
            break

In [20]:
thread_small_llama = threading.Thread(target=anonimized, args=(llm_small_llama, "small_llama" ))
thread_big_llama = threading.Thread(target=anonimized, args=(llm_big_llama, "big_llama" ))
thread_haiku = threading.Thread(target=anonimized, args=(llm_haiku, "haiku" ))
thread_sonet = threading.Thread(target=anonimized, args=(llm_sonet, "sonet" ))
thread_mistral = threading.Thread(target=anonimized, args=(llm_mistral, "mistral" ))

In [22]:
thread_small_llama.start()
thread_big_llama.start()
thread_haiku.start()
thread_sonet.start()

Folder 'data/anon/raw/haiku' created successfully
Folder 'data/anon/raw/small_llama' created successfully
Folder 'data/anon/raw/sonet' created successfully
Folder 'data/anon/raw/big_llama' created successfully
Folder 'data/anon/raw/sonet' created successfully
